In [ ]:
# import os
# import re
import numpy as np
import nltk
import pandas as pd
import collections
import matplotlib.pyplot as plt
import collections
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive
drive.mount("/content/drive/")
df=pd.read_csv("drive/My Drive/NLP summer Internship/Cleanned_Complaints.csv")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# Show label distribution for the whole original dataset
df=df[df["Label"]!="Other financial service"]
df=df.dropna()
df.groupby("Label")["Complaint"].count()

Label
Bank account or service                                                          14885
Checking or savings account                                                      22432
Consumer Loan                                                                     9473
Credit card or prepaid card                                                      58842
Credit reporting, credit repair services, or other personal consumer reports    212844
Debt collection                                                                 116674
Money transfer, virtual currency, or money service                               10691
Mortgage                                                                         66255
Payday loan, title loan, or personal loan                                         9189
Student loan                                                                     26410
Vehicle loan or lease                                                             9486
Name: Complaint, dtype: int64

In [ ]:
# Prepare three vendor dataset and one test set
df_all=df
df_wells=df[df["Company"]=="WELLS FARGO & COMPANY"]
credit_unions=["TRANSUNION INTERMEDIATE HOLDINGS, INC."]
banks=["CITIBANK, N.A.","BANK OF AMERICA, NATIONAL ASSOCIATION","JPMORGAN CHASE & CO.","U.S. BANCORP"]
mortgages=["Ocwen Financial Corporation","NATIONSTAR MORTGAGE","Ditech Financial LLC","SELECT PORTFOLIO SERVICING, INC.",
           "Specialized Loan Servicing Holdings LLC","SYNCHRONY FINANCIAL"]
balanced=["Santander Consumer USA Holdings Inc.","Ocwen Financial Corporation","BANK OF AMERICA, NATIONAL ASSOCIATION",
          "JPMORGAN CHASE & CO.","U.S. BANCORP","UNITED SERVICES AUTOMOBILE ASSOCIATION","SUNTRUST BANKS, INC.",
          "CITIZENS FINANCIAL GROUP, INC.","REGIONS FINANCIAL CORPORATION"]
df_cu=df[df["Company"].isin(credit_unions)]
df_banks=df[df["Company"].isin(banks)]
df_mortgages=df[df["Company"].isin(mortgages)]
df_balanced=df[df["Company"].isin(balanced)]

In [ ]:
# Corpus of the four dataset
cu_complaint=df_cu["Complaint_reduced"]
banks_complaint=df_banks["Complaint_reduced"]
mortgages_complaint=df_mortgages["Complaint_reduced"]
wells_complaint=df_wells["Complaint_reduced"]
balanced_complaint=df_balanced["Complaint_reduced"]
all_complaint=df_all["Complaint_reduced"]
y_wells=df_wells["Label"]
y_cu=df_cu["Label"]
y_banks=df_banks["Label"]
y_mortgages=df_mortgages["Label"]
y_balanced=df_balanced["Label"]
y_all=df_all["Label"]

In [ ]:
df[df["Label"]=="Bank account or service"].groupby("Company")["Label"].count().sort_values(ascending=False)[:20]

Company
BANK OF AMERICA, NATIONAL ASSOCIATION     2399
WELLS FARGO & COMPANY                     1927
JPMORGAN CHASE & CO.                      1650
CITIBANK, N.A.                            1329
U.S. BANCORP                               614
PNC Bank N.A.                              551
TD BANK US HOLDING COMPANY                 452
SUNTRUST BANKS, INC.                       397
CAPITAL ONE FINANCIAL CORPORATION          395
UNITED SERVICES AUTOMOBILE ASSOCIATION     342
Paypal Holdings, Inc                       342
CITIZENS FINANCIAL GROUP, INC.             293
BB&T CORPORATION                           271
REGIONS FINANCIAL CORPORATION              252
NAVY FEDERAL CREDIT UNION                  216
BBVA FINANCIAL CORPORATION                 195
FIFTH THIRD FINANCIAL CORPORATION          175
KEYCORP                                    169
SCOTTRADE BANK                             164
SANTANDER BANK, NATIONAL ASSOCIATION       153
Name: Label, dtype: int64

In [ ]:
print(cu_complaint.shape,banks_complaint.shape,mortgages_complaint.shape,balanced_complaint.shape,wells_complaint.shape,all_complaint.shape)

(55056,) (58302,) (29905,) (56618,) (17613,) (557181,)


In [ ]:
# from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from xgboost import XGBClassifier

Count Vectorizer method - RF

In [ ]:
CU_count_vectorizer=CountVectorizer(max_features=2000)
X_cu=CU_count_vectorizer.fit_transform(cu_complaint)
X_cu_wells=CU_count_vectorizer.transform(wells_complaint)
cu_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
cu_classifier.fit(X_cu, y_cu)
cu_predict=cu_classifier.predict(X_cu_wells)
print(classification_report(y_wells,cu_predict))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.00      0.00      0.00      1927
                                                 Checking or savings account       0.00      0.00      0.00      2635
                                                               Consumer Loan       0.00      0.00      0.00       536
                                                 Credit card or prepaid card       0.53      0.03      0.05      1990
Credit reporting, credit repair services, or other personal consumer reports       0.08      0.96      0.14      1400
                                                             Debt collection       0.38      0.07      0.12       857
                          Money transfer, virtual currency, or money service       0.00      0.00      0.00       587
                                                       

In [ ]:
Banks_count_vectorizer=CountVectorizer(max_features=2000)
X_banks=Banks_count_vectorizer.fit_transform(banks_complaint)
X_banks_wells=Banks_count_vectorizer.transform(wells_complaint)
banks_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
banks_classifier.fit(X_banks, y_banks)
banks_predict=banks_classifier.predict(X_banks_wells)
accuracy_score(y_wells,banks_predict)
print(classification_report(y_wells,banks_predict))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.61      0.28      0.38      1927
                                                 Checking or savings account       0.61      0.64      0.62      2635
                                                               Consumer Loan       0.39      0.14      0.20       536
                                                 Credit card or prepaid card       0.62      0.73      0.67      1990
Credit reporting, credit repair services, or other personal consumer reports       0.51      0.53      0.52      1400
                                                             Debt collection       0.43      0.60      0.50       857
                          Money transfer, virtual currency, or money service       0.48      0.70      0.57       587
                                                       

In [ ]:
Mortgages_count_vectorizer=CountVectorizer(max_features=2000)
X_mortgages=Mortgages_count_vectorizer.fit_transform(mortgages_complaint)
X_mortgages_wells=Mortgages_count_vectorizer.transform(wells_complaint)
mortgages_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
mortgages_classifier.fit(X_mortgages, y_mortgages)
mortgages_predict=mortgages_classifier.predict(X_mortgages_wells)
print(classification_report(y_wells,mortgages_predict))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.35      0.01      0.01      1927
                                                 Checking or savings account       0.50      0.02      0.04      2635
                                                               Consumer Loan       0.26      0.03      0.05       536
                                                 Credit card or prepaid card       0.40      0.77      0.53      1990
Credit reporting, credit repair services, or other personal consumer reports       0.46      0.54      0.50      1400
                                                             Debt collection       0.31      0.61      0.41       857
                          Money transfer, virtual currency, or money service       1.00      0.01      0.01       587
                                                       

In [ ]:
Balanced_count_vectorizer=CountVectorizer(max_features=2000)
X_balanced=Balanced_count_vectorizer.fit_transform(balanced_complaint)
X_balanced_wells=Balanced_count_vectorizer.transform(wells_complaint)
balanced_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
balanced_classifier.fit(X_balanced, y_balanced)
balanced_predict=balanced_classifier.predict(X_balanced_wells)
print(classification_report(y_wells,balanced_predict))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.59      0.37      0.45      1927
                                                 Checking or savings account       0.63      0.63      0.63      2635
                                                               Consumer Loan       0.50      0.18      0.27       536
                                                 Credit card or prepaid card       0.65      0.74      0.69      1990
Credit reporting, credit repair services, or other personal consumer reports       0.52      0.54      0.53      1400
                                                             Debt collection       0.46      0.56      0.50       857
                          Money transfer, virtual currency, or money service       0.50      0.69      0.58       587
                                                       

In [ ]:
All_count_vectorizer=CountVectorizer(max_features=2000)
X_all=All_count_vectorizer.fit_transform(all_complaint)
X_all_wells=All_count_vectorizer.transform(wells_complaint)

In [ ]:
all_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
all_classifier.fit(X_all, y_all)
all_predict=all_classifier.predict(X_all_wells)
print(classification_report(y_wells,all_predict))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.78      0.63      0.70      1927
                                                 Checking or savings account       0.78      0.76      0.77      2635
                                                               Consumer Loan       0.83      0.59      0.69       536
                                                 Credit card or prepaid card       0.73      0.78      0.75      1990
Credit reporting, credit repair services, or other personal consumer reports       0.62      0.46      0.53      1400
                                                             Debt collection       0.57      0.58      0.57       857
                          Money transfer, virtual currency, or money service       0.56      0.84      0.67       587
                                                       

TFIDF vectorizer - RF

In [ ]:
CU_tfidf_vectorizer=TfidfVectorizer(max_features=2000)
X_cu_tfidf=CU_tfidf_vectorizer.fit_transform(cu_complaint)
X_cu_wells_tfidf=CU_tfidf_vectorizer.transform(wells_complaint)
cu_tfidf_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
cu_tfidf_classifier.fit(X_cu_tfidf, y_cu)
cu_predict_tfidf=cu_tfidf_classifier.predict(X_cu_wells_tfidf)
print(classification_report(y_wells,cu_predict_tfidf))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.00      0.00      0.00      1927
                                                 Checking or savings account       0.00      0.00      0.00      2635
                                                               Consumer Loan       0.00      0.00      0.00       536
                                                 Credit card or prepaid card       0.62      0.02      0.04      1990
Credit reporting, credit repair services, or other personal consumer reports       0.08      0.98      0.15      1400
                                                             Debt collection       0.63      0.06      0.10       857
                          Money transfer, virtual currency, or money service       0.00      0.00      0.00       587
                                                       

In [ ]:
Banks_tfidf_vectorizer=TfidfVectorizer(max_features=2000)
X_banks_tfidf=Banks_tfidf_vectorizer.fit_transform(banks_complaint)
X_banks_wells_tfidf=Banks_tfidf_vectorizer.transform(wells_complaint)
banks_tfidf_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
banks_tfidf_classifier.fit(X_banks_tfidf, y_banks)
banks_predict_tfidf=banks_tfidf_classifier.predict(X_banks_wells_tfidf)
print(classification_report(y_wells,banks_predict_tfidf))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.62      0.28      0.39      1927
                                                 Checking or savings account       0.60      0.65      0.62      2635
                                                               Consumer Loan       0.43      0.11      0.17       536
                                                 Credit card or prepaid card       0.64      0.76      0.69      1990
Credit reporting, credit repair services, or other personal consumer reports       0.53      0.57      0.55      1400
                                                             Debt collection       0.46      0.61      0.53       857
                          Money transfer, virtual currency, or money service       0.49      0.72      0.59       587
                                                       

In [ ]:
Mortgages_tfidf_vectorizer=TfidfVectorizer(max_features=2000)
X_mortgages_tfidf=Mortgages_tfidf_vectorizer.fit_transform(mortgages_complaint)
X_mortgages_wells_tfidf=Mortgages_tfidf_vectorizer.transform(wells_complaint)
mortgages_tfidf_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
mortgages_tfidf_classifier.fit(X_mortgages_tfidf, y_mortgages)
mortgages_predict_tfidf=mortgages_tfidf_classifier.predict(X_mortgages_wells_tfidf)
print(classification_report(y_wells,mortgages_predict_tfidf))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.39      0.02      0.03      1927
                                                 Checking or savings account       0.51      0.01      0.03      2635
                                                               Consumer Loan       0.32      0.02      0.03       536
                                                 Credit card or prepaid card       0.37      0.79      0.50      1990
Credit reporting, credit repair services, or other personal consumer reports       0.48      0.62      0.54      1400
                                                             Debt collection       0.34      0.62      0.44       857
                          Money transfer, virtual currency, or money service       1.00      0.00      0.01       587
                                                       

In [ ]:
Balanced_tfidf_vectorizer=TfidfVectorizer(max_features=2000)
X_balanced_tfidf=Balanced_tfidf_vectorizer.fit_transform(balanced_complaint)
X_balanced_wells_tfidf=Balanced_tfidf_vectorizer.transform(wells_complaint)
balanced_tfidf_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
balanced_tfidf_classifier.fit(X_balanced_tfidf, y_balanced)
balanced_predict_tfidf=balanced_tfidf_classifier.predict(X_balanced_wells_tfidf)
print(classification_report(y_wells,balanced_predict_tfidf))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.60      0.37      0.46      1927
                                                 Checking or savings account       0.62      0.64      0.63      2635
                                                               Consumer Loan       0.53      0.19      0.28       536
                                                 Credit card or prepaid card       0.65      0.76      0.70      1990
Credit reporting, credit repair services, or other personal consumer reports       0.52      0.59      0.55      1400
                                                             Debt collection       0.49      0.58      0.53       857
                          Money transfer, virtual currency, or money service       0.51      0.67      0.58       587
                                                       

In [ ]:
All_tfidf_vectorizer=TfidfVectorizer(max_features=2000)
X_all_tfidf=All_tfidf_vectorizer.fit_transform(all_complaint)
X_all_wells_tfidf=All_tfidf_vectorizer.transform(wells_complaint)

In [ ]:
all_tfidf_classifier=RandomForestClassifier(max_depth=20,n_estimators=300, random_state=0, n_jobs=-1, class_weight='balanced_subsample')
all_tfidf_classifier.fit(X_all_tfidf, y_all)
all_predict_tfidf=all_tfidf_classifier.predict(X_all_wells_tfidf)
print(classification_report(y_wells,all_predict_tfidf))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.82      0.67      0.74      1927
                                                 Checking or savings account       0.79      0.78      0.78      2635
                                                               Consumer Loan       0.88      0.64      0.74       536
                                                 Credit card or prepaid card       0.74      0.79      0.76      1990
Credit reporting, credit repair services, or other personal consumer reports       0.64      0.49      0.56      1400
                                                             Debt collection       0.58      0.55      0.57       857
                          Money transfer, virtual currency, or money service       0.56      0.82      0.67       587
                                                       